<h1>Data Processing</h1>

<h3>Importpath</h3>

Add Projectfolder to Importpath

In [1]:
import sys
import os
cwd = os.path.abspath(".")
sys.path.insert(0, cwd.replace("Notebook", "Code"))

sys.path

['/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Code',
 '/home/avatar/Dokumente/Projekte/Python/EconomicCalendarImpactAnalysis/Notebook',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python310.zip',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/lib-dynload',
 '',
 '/home/avatar/.local/lib/python3.10/site-packages',
 '/home/avatar/.pyenv/versions/3.10.13/lib/python3.10/site-packages']

<h3>Imports</h3>

In [2]:
import pandas as pd

import fmp
import influx as db

<h3>Data Processing</h3>

In [ ]:
start_date = pd.Timestamp("2024-01-01")
stop_date = pd.Timestamp("2024-01-07")

FMP = fmp.FinancialModelingPrep(deployment = "linode")
FMP.economic_calendar_pipeline(start_date, stop_date)

In [3]:
FMP = fmp.FinancialModelingPrep(deployment = "linode")
FMP.economic_calendar_pipeline_longrange()

Piping Economic Calendar from 2023-01-01 00:00:00 to 2023-01-31 00:00:00
Getting Economic Calendar
Parsing Economic Calendar
Preprocessing Economic Calendar
Ingesting Economic Calendar
Counter: 1, Start at: 0, Stop at: 1721
Piping Economic Calendar from 2023-01-31 00:00:00 to 2023-03-02 00:00:00
Getting Economic Calendar
Parsing Economic Calendar
Preprocessing Economic Calendar
Ingesting Economic Calendar
Counter: 1, Start at: 0, Stop at: 1914
Piping Economic Calendar from 2023-03-02 00:00:00 to 2023-04-01 00:00:00
Getting Economic Calendar
Parsing Economic Calendar
Preprocessing Economic Calendar
Ingesting Economic Calendar
Counter: 1, Start at: 0, Stop at: 1623
Piping Economic Calendar from 2023-04-01 00:00:00 to 2023-05-01 00:00:00
Getting Economic Calendar
Parsing Economic Calendar
Preprocessing Economic Calendar
Ingesting Economic Calendar
Counter: 1, Start at: 0, Stop at: 1704
Piping Economic Calendar from 2023-05-01 00:00:00 to 2023-05-31 00:00:00
Getting Economic Calendar
Parsi

<h3>Test new Configuration for InfluxDB</h3>

In [ ]:
import influx as db

db.InfluxDatabase()

<h3>Data Processing: Pricedata form MetaTrader5 CSV's to InfluxDB</h3>

In [ ]:
import influx
import mt5

db = influx.InfluxDatabase(deployment = "linode")
mt5 = mt5.MetaTrader5()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]

major = majors[6]

print("Step #1")
raw_data = mt5.load_csv_to_dataframe(file_name=f"MetaTrader5 {major} 1MIN Full Year 2023.csv")
print("Step #2")
clean_data = mt5.preprocess_csv_dataframe(raw_data, symbol = major)

print(clean_data)

print("Step #3")
db.ingest_data(clean_data, mode="live")


<h3>Data Quering: InfluxDB Pricedata<h3>

In [ ]:
import influx
import pandas as pd

db = influx.InfluxDatabase()

majors = ["AUDUSD", "EURUSD", "GBPUSD", "NZDUSD", "USDCAD", "USDCHF", "USDJPY"]
major = majors[1]

time1 = pd.Timestamp("2023-06-23T16:12")

#AUDUSD 2023.06.23	16:12:00

data1 = db.query_data(time = time1, symbol = major, timeframe = "1min", source = "MetaTrader5")

print(data1)